# Solution description

## Task

The task is to build an algorithm for prohibitory traffic sign recognition.

## Data

### Data expanding

Twenty images are provided as training set for building and fine-tuning a model.Taking screenshots or using google-streetview Python module are suggested to add more images, however, I expanded the size of the set by augmenting the data.
For augmentation, three methods were used:
<ol>
  <li>Flipping <i>(20 images became 40)</i>.</li>
  <li>Blurring <i>(for the 40 images resulted from 1)</i>.</li>
  <li>Adding noise with increasing brightness <i>(for the 40 images resulted from 1)</i>.</li>
</ol>

Accordingly, the overall size of the set is 120 images (*split: 70 train, 10 validation and 40 test*). <br>
A note to keep in mind here, that augmentation has some drawbacks:
<ul>
  <li>It might cause model overfitting as a result of instances duplication.</li>
  <li>The overll number of images might not be enough.</li>
  <li>In case of traffic scene/signs, some augmentation methods might corrupt the data, e.g. Vertical flipping, rotation (with angle), etc.</li>
</ul>

On the other hand, data augmentation is a fast way to expand data (e.g. double and tripple) with the minimum amount of effort compared to other manual methods like: screenshorts collecting.

Data consists of six unbalanced distributed categories:
<ul>
  <li>A: No turn</li>
  <li>B: Speed limit</li>
  <li>C: Road closed</li>
  <li>D: No entry</li>
  <li>E: No stop/park</li>
  <li>F: Others</li>
</ul>

### Images annotation

Since the provided data is not from a standard dataset, annotation was missing. To solve this, I used online tool (*www.makesense.ai*) for manual annotation. The tool proivdes option to choose the proper format for the annotations, i.e. YOLO, VOX XML and CSV.

## Model

To tackle this task, I trained the infamous object detection and recognition model YOLOv5 on the traffic signs data using its pretraining weights. To do so, I followed the available instruction on its implementation repository (https://github.com/ultralytics/yolov5).

# Import packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from skimage.util import random_noise

# Data augmentation

## Define provided images directory

In [ ]:
data_dir = './training_images/' # provided images

### Flip

In [ ]:
for i in range(1, 21):
    im_path = f'./training_images/{i}.jpg'
    img = cv2.imread(im_path)
    
    img = cv2.flip(img, 1)
    
    cv2.imwrite(f'./training_images/{i+20}.jpg', img)

### Blur

In [ ]:
for i in range(1, 41):
    im_path = f'./training_images/{i}.jpg'
    img = cv2.imread(im_path)
    
    img = cv2.blur(img, (5, 5))
    
    cv2.imwrite(f'./training_images/{i+40}.jpg', img)

### Random noise and increase brightness

In [ ]:
def increaseBrightness(img, value=30):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

for i in range(1,41):
    img_path = f'./training_images/{i}.jpg'
    img = cv2.imread(img_path)
    
    img = increaseBrightness(img, value=70)
    
    img = random_noise(img, mode='s&p',amount=0.05)
    img = np.array(255*img, dtype = 'uint8')
    
    cv2.imwrite(f'./training_images/{i+80}.jpg', img)

# Images annotation

## Labels

In [ ]:
labels = {
    0 : 'a_no_turn',
    1 : 'b_speed_limit',
    2 : 'c_road_closed',
    3 : 'd_no_entry',
    4 : 'no_stop_park',
    5 : 'f_others'
    }

# YOLOv5 training

## Install requirements

In [ ]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 14566, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 14566 (delta 84), reused 116 (delta 71), pack-reused 14430
Receiving objects: 100% (14566/14566), 13.50 MiB | 16.05 MiB/s, done.
Resolving deltas: 100% (10066/10066), done.
/content/yolov5
     |████████████████████████████████| 1.6 MB 38.1 MB/s 
     |████████████████████████████████| 145 kB 48.8 MB/s 
     |████████████████████████████████| 178 kB 65.5 MB/s 
     |████████████████████████████████| 67 kB 6.7 MB/s 
     |████████████████████████████████| 54 kB 1.6 MB/s 
     |████████████████████████████████| 138 kB 68.5 MB/s 
     |████████████████████████████████| 62 kB 1.8 MB/s 
Setup complete. Using torch 1.12.1+cu113 (Tesla T4)


## Setup environment and prepare data

In [ ]:
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

### Define dataset location in drive

In [ ]:
dataset_location = '/content/drive/MyDrive/PhD/AI_in_DS/datasets/traffic_signs'

### Move data to proper directroy

In [ ]:
!cp '{dataset_location}/data_traffic_signs.yaml' 'data/data_traffic_signs.yaml'
!cp '{dataset_location}/yolov5s_traffic_signs.yaml' 'models/yolov5s_traffic_signs.yaml'
!mkdir -p '/content/yolov5/datasets'
!cp -ar '{dataset_location}' '/content/yolov5/datasets/traffic_signs'

## Train custom YOLOv5 model

In [ ]:
!python train.py --batch 16 --epochs 320 --data 'data/data_traffic_signs.yaml' --cfg 'models/yolov5s_traffic_signs.yaml' --weights yolov5s.pt --cache --freeze 12 --project 'traffic_signs_runs' --name 'feature_extraction'

train: weights=yolov5s.pt, cfg=models/yolov5s_traffic_signs.yaml, data=data/data_traffic_signs.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=320, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=traffic_signs_runs, name=feature_extraction, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[12], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-216-g6e544d5 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, 

### Finetune

In [ ]:
!python train.py --hyp 'data/hyps/hyp.VOC.yaml' --batch 16 --epochs 100 --data 'data/data_traffic_signs.yaml' --weights '/content/yolov5/traffic_signs_runs/feature_extraction/weights/best.pt' --project 'traffic_signs_runs' --name 'finetune' --cache

train: weights=/content/yolov5/traffic_signs_runs/feature_extraction/weights/best.pt, cfg=, data=data/data_traffic_signs.yaml, hyp=data/hyps/hyp.VOC.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=traffic_signs_runs, name=finetune, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-216-g6e544d5 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.00334, lrf=0.15135, momentum=0.74832, weight_decay=0.00025, warmup_epochs=3.3835, warmup_momentum=0.59462, warmup_bias_lr=0.18657, box=0.02, cls=0.21638, cl

### Evaluate

In [ ]:
!python val.py --batch 64 --data 'data/data_traffic_signs.yaml' --weights '/content/yolov5/traffic_signs_runs/finetune/weights/best.pt' --task test --project 'traffic_signs_runs' --name 'validation' --augment

val: data=data/data_traffic_signs.yaml, weights=['/content/yolov5/traffic_signs_runs/finetune/weights/best.pt'], batch_size=64, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=traffic_signs_runs, name=validation, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.2-216-g6e544d5 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 157 layers, 7026307 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning '/content/yolov5/datasets/traffic_signs/labels/test' images and labels...40 found, 0 missing, 0 empty, 0 corrupt: 100% 40/40 [00:00<00:00, 304.36it/s]
test: New cache created: /content/yolov5/datasets/traffic_signs/labels/test.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 1/1 [00:01<00:00,  1.33s/it]
                   all         40 

### Infere

In [ ]:
!python detect.py --sourc '/content/yolov5/datasets/traffic_signs/images/test' --weights '/content/yolov5/traffic_signs_runs/finetune/weights/best.pt' --conf 0.6 --iou 0.45 --augment --project 'traffic_signs_runs' --name 'detect_test'

detect: weights=['/content/yolov5/traffic_signs_runs/finetune/weights/best.pt'], source=/content/yolov5/datasets/traffic_signs/images/test, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.6, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=True, visualize=False, update=False, project=traffic_signs_runs, name=detect_test, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-216-g6e544d5 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 157 layers, 7026307 parameters, 0 gradients, 15.8 GFLOPs
image 1/40 /content/yolov5/datasets/traffic_signs/images/test/1.jpg: 320x640 1 b_speed_limit, 1 c_road_closed, 35.5ms
image 2/40 /content/yolov5/datasets/traffic_signs/images/test/10.jpg: 320x640 1 b_speed_limit, 1 c_road_closed, 1 e_no_stop_park, 25.2ms
image 3/40 /conten

### Export

In [ ]:
!python export.py --weights '/content/yolov5/traffic_signs_runs/finetune/weights/best.pt' --include engine onnx --data 'data/data_traffic_signs.yaml' --device 0 --imgsz 640 640

export: data=data/data_traffic_signs.yaml, weights=['/content/yolov5/traffic_signs_runs/finetune/weights/best.pt'], imgsz=[640, 640], batch_size=1, device=0, half=False, inplace=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=12, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['engine', 'onnx']
YOLOv5 🚀 v6.2-216-g6e544d5 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 157 layers, 7026307 parameters, 0 gradients, 15.8 GFLOPs

PyTorch: starting from /content/yolov5/traffic_signs_runs/finetune/weights/best.pt with output shape (1, 25200, 11) (13.7 MB)
requirements: YOLOv5 requirement "nvidia-tensorrt" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.ngc.nvidia.com, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for nvidia-cublas-cu11: filename=nvidia_cublas_cu11-2022.4.8-py3-non